In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import pandas as pd
import os
target_path = '../data/LOG_G/stable/'
contents = os.listdir(target_path)
data = pd.DataFrame()
num = 0
for c in range(len(contents)):
    print(target_path+contents[c])
    n_data = pd.read_csv(target_path+contents[c],header=None)
    data = pd.concat([data,n_data])

print(len(data))
d = data
#print(ball)
#print(joy)

../data/LOG_G/stable/log201807311344.csv
../data/LOG_G/stable/log201807292107.csv
../data/LOG_G/stable/log201807292213.csv
../data/LOG_G/stable/log201807311352.csv
../data/LOG_G/stable/log201807311349.csv
../data/LOG_G/stable/log201807292116.csv
../data/LOG_G/stable/log201807311336.csv
../data/LOG_G/stable/log201807292215.csv
../data/LOG_G/stable/log201807292207.csv
../data/LOG_G/stable/log201807292110.csv
../data/LOG_G/stable/log201807311347.csv
32853


In [3]:
d.columns = ['l_bx','l_by','l_pole0x','l_pole0y','l_pole1x','l_pole1y','g_bx','g_by','g_pole0x','g_pole0y','g_pole1x','g_pole1y','g_px','g_py','g_pth','ljx','ljy','rjx','rjy']
#d = d[d.bx!=-1]

In [4]:
'''
from sklearn.cross_validation import train_test_split
t1 = []
t2 = []
def f1(x,y):
    return y * np.sin(x) + np.square(x)
def f2(x,y):
    return x * np.cos(y) + np.square(y)
for a in range(1000):
    r = np.array([np.random.rand(), np.random.rand()])
    t1.append(r)
    t2.append(np.array([ f1(r[0],r[1]),f2(r[0],r[1]) ]))
IN_train, IN_test, TAR_train, TAR_test = train_test_split(t1,t2,test_size=0.2,random_state=0)

IN_train = np.array(IN_train)
IN_test = np.array(IN_test)
TAR_train = np.array(TAR_train)
TAR_test = np.array(TAR_test)
print(IN_train[0])
print(IN_test[0])
print(TAR_train[0])
print(TAR_test[0])
print(f1(1,1))
print(f2(1,1))
'''

'\nfrom sklearn.cross_validation import train_test_split\nt1 = []\nt2 = []\ndef f1(x,y):\n    return y * np.sin(x) + np.square(x)\ndef f2(x,y):\n    return x * np.cos(y) + np.square(y)\nfor a in range(1000):\n    r = np.array([np.random.rand(), np.random.rand()])\n    t1.append(r)\n    t2.append(np.array([ f1(r[0],r[1]),f2(r[0],r[1]) ]))\nIN_train, IN_test, TAR_train, TAR_test = train_test_split(t1,t2,test_size=0.2,random_state=0)\n\nIN_train = np.array(IN_train)\nIN_test = np.array(IN_test)\nTAR_train = np.array(TAR_train)\nTAR_test = np.array(TAR_test)\nprint(IN_train[0])\nprint(IN_test[0])\nprint(TAR_train[0])\nprint(TAR_test[0])\nprint(f1(1,1))\nprint(f2(1,1))\n'

In [5]:
from sklearn.cross_validation import train_test_split
l_ball = np.array(d.iloc[:,0:2])
l_pole0 = np.array(d.iloc[:,2:4])
l_pole1 = np.array(d.iloc[:,4:6])
input_d = np.array(d.iloc[:,0:6])
g_ball = np.array(d.iloc[:,6:8])
g_pole0 = np.array(d.iloc[:,8:10])
g_pole1 = np.array(d.iloc[:,10:12])
g_pos = np.array(d.iloc[:,12:14])
output_d = np.array(d.iloc[:,14:18])
ljoy = np.array(d.iloc[:,14:16])
rjoy = np.array(d.iloc[:,16:18])

input_d = np.array(d.iloc[:,0:6])
output_d = np.array(d.iloc[:,14:18])
print(input_d.shape)
print(output_d.shape)

IN_train, IN_test, TAR_train, TAR_test = train_test_split(input_d,output_d,test_size=0.2,random_state=0)
#X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)
#IN_train = IN_train/100.0
#IN_test = IN_test/100.0
for t in range(10):
    print(IN_train[t])
'''
print(IN_train[0])
print(IN_test[0])
print(TAR_train[0])
print(TAR_test[0])
'''

(32853, 6)
(32853, 4)
[ 0.26507283 -0.12912751  1.93868955  2.07983174  2.28975789 -0.49635751]
[ 0.7522718   0.32540024  1.74107202  2.07986238  2.89844302 -0.24833272]
[ 0.40731088 -0.01143569  1.49269182  0.32712971  1.22747513 -2.25930801]
[2.82947013 1.27166372 1.00351973 3.19039456 3.31746766 2.00479555]
[ 3.10863438 -0.73276001  6.68595478  1.98437951  7.59210989 -0.45260283]
[2.67322077 1.01560967 1.01623561 3.08201294 3.22152338 1.7048035 ]
[ 0.44459806  0.38500604  1.49200801  1.93614835  2.3476051  -0.51904077]
[ 0.52533711 -0.0316782   0.88645154  0.96624206  3.01853728 -0.52178025]
[ 0.54904566  0.07638845  1.39258296  1.52994657  1.75798521 -1.04424868]
[2.39541926 0.03300046 3.05722124 2.59769167 3.27572722 0.00688965]


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


'\nprint(IN_train[0])\nprint(IN_test[0])\nprint(TAR_train[0])\nprint(TAR_test[0])\n'

In [6]:
#　初期化
sess = tf.Session()

IN_size = 6
TAR_size = 4
INPUT = tf.placeholder(shape=[None, IN_size], dtype=tf.float32)
TARGET = tf.placeholder(shape=[None, TAR_size], dtype=tf.float32)

In [7]:
hidden_size = [50,10]
batch_size = 100
iter_size = 100000

#　隠れ層の作成 （last layerに直結）
weights = []
biases = []
layers = []
tmp_size = IN_size
last_layer = INPUT
for hsize in hidden_size:
    weights.append(tf.Variable(tf.random_normal(shape=[tmp_size, hsize], stddev=10.0)))
    biases.append(tf.Variable(tf.random_normal(shape=[hsize], stddev=10.0)))
    layers.append(tf.nn.relu(tf.add(tf.matmul(last_layer, weights[-1]), biases[-1]))) # 活性化関数
    tmp_size = hsize
    last_layer = layers[-1]
    print(layers[-1])
#　出力層
weights.append(tf.Variable(tf.random_normal(shape=[tmp_size, TAR_size], stddev=10.0)))
biases.append(tf.Variable(tf.random_normal(shape=[TAR_size], stddev=10.0)))
layers.append(tf.add(tf.matmul(last_layer, weights[-1]), biases[-1]))
final_output = layers[-1]
print(final_output)

Tensor("Relu:0", shape=(?, 50), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10), dtype=float32)
Tensor("Add_2:0", shape=(?, 4), dtype=float32)


In [ ]:
#　学習
loss = tf.losses.mean_squared_error(TARGET,final_output)
opt = tf.train.AdamOptimizer(0.001)
#opt = tf.train.GradientDescentOptimizer(0.01)
train_step = opt.minimize(loss)
init = tf.global_variables_initializer()
sess.run(init)
loss_vec=[]
for i in range(iter_size):
    rand_index = np.random.choice(len(IN_train), size=batch_size,replace=False)
    rand_in = IN_train[rand_index]
    rand_tar = TAR_train[rand_index]
    sess.run(train_step, feed_dict={INPUT:rand_in, TARGET: rand_tar})
    loss_vec.append(sess.run(loss, feed_dict={INPUT:rand_in, TARGET: rand_tar}))
    if (i+1)%25==0:
#        print(sess.run(INPUT, feed_dict={INPUT:rand_in, TARGET: rand_tar}))
        print('Generation:'+str(i+1)+', Loss = '+str(loss_vec[-1]))

Generation:25, Loss = 1834169300.0
Generation:50, Loss = 1596455200.0
Generation:75, Loss = 1822894600.0
Generation:100, Loss = 1717606800.0
Generation:125, Loss = 1676933600.0
Generation:150, Loss = 1364489200.0
Generation:175, Loss = 1391657700.0
Generation:200, Loss = 1463875300.0
Generation:225, Loss = 1155676400.0
Generation:250, Loss = 1323522700.0
Generation:275, Loss = 1051876100.0
Generation:300, Loss = 1378272000.0
Generation:325, Loss = 1252575000.0
Generation:350, Loss = 986223200.0
Generation:375, Loss = 963811500.0
Generation:400, Loss = 1086933800.0
Generation:425, Loss = 824237600.0
Generation:450, Loss = 856738200.0
Generation:475, Loss = 812469600.0
Generation:500, Loss = 779717760.0
Generation:525, Loss = 860124860.0
Generation:550, Loss = 810797440.0
Generation:575, Loss = 809269440.0
Generation:600, Loss = 667693300.0
Generation:625, Loss = 683129700.0
Generation:650, Loss = 713663900.0
Generation:675, Loss = 556631000.0
Generation:700, Loss = 616279700.0
Generatio

Generation:5925, Loss = 11694630.0
Generation:5950, Loss = 8442949.0
Generation:5975, Loss = 12440559.0
Generation:6000, Loss = 11373980.0
Generation:6025, Loss = 8005908.5
Generation:6050, Loss = 7521499.5
Generation:6075, Loss = 7623749.0
Generation:6100, Loss = 9070705.0
Generation:6125, Loss = 15971505.0
Generation:6150, Loss = 8676105.0
Generation:6175, Loss = 8264035.0
Generation:6200, Loss = 6914278.5
Generation:6225, Loss = 8296430.5
Generation:6250, Loss = 9265450.0
Generation:6275, Loss = 7320260.5
Generation:6300, Loss = 8618029.0
Generation:6325, Loss = 7741430.5
Generation:6350, Loss = 6469093.0
Generation:6375, Loss = 7922189.0
Generation:6400, Loss = 6938733.0
Generation:6425, Loss = 7772744.5
Generation:6450, Loss = 7131204.5
Generation:6475, Loss = 8401585.0
Generation:6500, Loss = 6477371.0
Generation:6525, Loss = 6772674.5
Generation:6550, Loss = 7930085.0
Generation:6575, Loss = 6185161.5
Generation:6600, Loss = 5623878.5
Generation:6625, Loss = 6445277.0
Generation

Generation:12000, Loss = 17645.56
Generation:12025, Loss = 6540.6587
Generation:12050, Loss = 908354.75
Generation:12075, Loss = 13626.326
Generation:12100, Loss = 16940.088
Generation:12125, Loss = 14237.97
Generation:12150, Loss = 11652.29
Generation:12175, Loss = 8011.8364
Generation:12200, Loss = 10692.277
Generation:12225, Loss = 17012.832
Generation:12250, Loss = 27526.955
Generation:12275, Loss = 19858.32
Generation:12300, Loss = 7019.225
Generation:12325, Loss = 1933.5203
Generation:12350, Loss = 44418.36
Generation:12375, Loss = 8201.813
Generation:12400, Loss = 21752.57
Generation:12425, Loss = 21595.568
Generation:12450, Loss = 2792.6838
Generation:12475, Loss = 52468.47
Generation:12500, Loss = 11026.171
Generation:12525, Loss = 52645.434
Generation:12550, Loss = 9265.356
Generation:12575, Loss = 12275.735
Generation:12600, Loss = 9578.098
Generation:12625, Loss = 14573.105
Generation:12650, Loss = 7758.0986
Generation:12675, Loss = 20056.178
Generation:12700, Loss = 9997.3

Generation:17925, Loss = 70.3337
Generation:17950, Loss = 70.155426
Generation:17975, Loss = 69.54709
Generation:18000, Loss = 69.694954
Generation:18025, Loss = 125.784065
Generation:18050, Loss = 69.19565
Generation:18075, Loss = 68.53332
Generation:18100, Loss = 68.160614
Generation:18125, Loss = 69.34227
Generation:18150, Loss = 68.06534
Generation:18175, Loss = 67.611755
Generation:18200, Loss = 67.3593
Generation:18225, Loss = 66.90844
Generation:18250, Loss = 92.31026
Generation:18275, Loss = 82.74746
Generation:18300, Loss = 66.47456
Generation:18325, Loss = 66.02311
Generation:18350, Loss = 65.53249
Generation:18375, Loss = 65.348305
Generation:18400, Loss = 65.141945
Generation:18425, Loss = 64.72149
Generation:18450, Loss = 64.541626
Generation:18475, Loss = 64.25824
Generation:18500, Loss = 64.06702
Generation:18525, Loss = 63.634697
Generation:18550, Loss = 63.494247
Generation:18575, Loss = 63.000076
Generation:18600, Loss = 62.833935
Generation:18625, Loss = 62.69327
Gen

Generation:23850, Loss = 17.333712
Generation:23875, Loss = 17.246492
Generation:23900, Loss = 17.15824
Generation:23925, Loss = 16.885098
Generation:23950, Loss = 16.887022
Generation:23975, Loss = 16.762384
Generation:24000, Loss = 16.55605
Generation:24025, Loss = 16.468477
Generation:24050, Loss = 16.347672
Generation:24075, Loss = 16.184786
Generation:24100, Loss = 16.04886
Generation:24125, Loss = 16.101137
Generation:24150, Loss = 15.901714
Generation:24175, Loss = 15.720416
Generation:24200, Loss = 15.741539
Generation:24225, Loss = 15.460588
Generation:24250, Loss = 15.435426
Generation:24275, Loss = 15.290318
Generation:24300, Loss = 15.193011
Generation:24325, Loss = 15.138852
Generation:24350, Loss = 14.739528
Generation:24375, Loss = 14.6929245
Generation:24400, Loss = 14.516823
Generation:24425, Loss = 14.485599
Generation:24450, Loss = 14.49356
Generation:24475, Loss = 14.287002
Generation:24500, Loss = 14.1368265
Generation:24525, Loss = 14.100875
Generation:24550, Loss

...

Generation:52800, Loss = 0.111216776
Generation:52825, Loss = 0.12410136
Generation:52850, Loss = 0.12601829
Generation:52875, Loss = 0.118690245
Generation:52900, Loss = 0.11757663
Generation:52925, Loss = 0.12725276
Generation:52950, Loss = 0.121989734
Generation:52975, Loss = 0.13380076
Generation:53000, Loss = 0.1421952
Generation:53025, Loss = 0.11702278
Generation:53050, Loss = 0.12269443
Generation:53075, Loss = 0.106250905
Generation:53100, Loss = 0.114589274
Generation:53125, Loss = 0.12803988
Generation:53150, Loss = 0.1114773
Generation:53175, Loss = 0.12887555
Generation:53200, Loss = 0.115302905
Generation:53225, Loss = 0.110794604
Generation:53250, Loss = 0.13170984
Generation:53275, Loss = 0.12185622
Generation:53300, Loss = 0.12460536
Generation:53325, Loss = 0.12678003
Generation:53350, Loss = 0.12287389
Generation:53375, Loss = 0.12581712
Generation:53400, Loss = 0.14217457
Generation:53425, Loss = 0.10553958
Generation:53450, Loss = 0.12367456
Generation:53475, Loss 

Generation:58475, Loss = 0.120722145
Generation:58500, Loss = 0.12348125
Generation:58525, Loss = 0.14509632
Generation:58550, Loss = 0.10622715
Generation:58575, Loss = 0.12583075
Generation:58600, Loss = 0.12532285
Generation:58625, Loss = 0.11786267
Generation:58650, Loss = 0.12687124
Generation:58675, Loss = 0.12934655
Generation:58700, Loss = 0.11837036
Generation:58725, Loss = 0.11541523
Generation:58750, Loss = 0.112351425
Generation:58775, Loss = 0.09819829
Generation:58800, Loss = 0.13130827
Generation:58825, Loss = 0.11353839
Generation:58850, Loss = 0.10728062
Generation:58875, Loss = 0.107883856
Generation:58900, Loss = 0.110446
Generation:58925, Loss = 0.12965176
Generation:58950, Loss = 0.11890617
Generation:58975, Loss = 0.1093407
Generation:59000, Loss = 0.124520734
Generation:59025, Loss = 0.12439977
Generation:59050, Loss = 0.11757541
Generation:59075, Loss = 0.14234327
Generation:59100, Loss = 0.12312667
Generation:59125, Loss = 0.12146595
Generation:59150, Loss = 0.

Generation:64175, Loss = 0.11517915
Generation:64200, Loss = 0.12253219
Generation:64225, Loss = 0.110983744
Generation:64250, Loss = 0.11667435
Generation:64275, Loss = 0.12546057
Generation:64300, Loss = 0.11761032
Generation:64325, Loss = 0.11560383
Generation:64350, Loss = 0.11195577
Generation:64375, Loss = 0.12370526
Generation:64400, Loss = 0.12904395
Generation:64425, Loss = 0.12085389
Generation:64450, Loss = 0.10290415
Generation:64475, Loss = 0.12143577
Generation:64500, Loss = 0.1276444
Generation:64525, Loss = 0.12222913
Generation:64550, Loss = 0.11835472
Generation:64575, Loss = 0.11357308
Generation:64600, Loss = 0.119253665
Generation:64625, Loss = 0.110152215
Generation:64650, Loss = 0.10155997
Generation:64675, Loss = 0.09764836
Generation:64700, Loss = 0.1316659
Generation:64725, Loss = 0.119965956
Generation:64750, Loss = 0.12735347
Generation:64775, Loss = 0.12240062
Generation:64800, Loss = 0.121892154
Generation:64825, Loss = 0.11006443
Generation:64850, Loss = 

Generation:69900, Loss = 0.11685751
Generation:69925, Loss = 0.123580545
Generation:69950, Loss = 0.12110901
Generation:69975, Loss = 0.111647025
Generation:70000, Loss = 0.11812404
Generation:70025, Loss = 0.13743638
Generation:70050, Loss = 0.11701701
Generation:70075, Loss = 0.09948147
Generation:70100, Loss = 0.12498984
Generation:70125, Loss = 0.12829588
Generation:70150, Loss = 0.12582268
Generation:70175, Loss = 0.11908367
Generation:70200, Loss = 0.12979639
Generation:70225, Loss = 0.10927124
Generation:70250, Loss = 0.11354165
Generation:70275, Loss = 0.12951493
Generation:70300, Loss = 0.12797493
Generation:70325, Loss = 0.11275377
Generation:70350, Loss = 0.11637932
Generation:70375, Loss = 0.13154787
Generation:70400, Loss = 0.122846
Generation:70425, Loss = 0.118329875
Generation:70450, Loss = 0.12244047
Generation:70475, Loss = 0.13333866
Generation:70500, Loss = 0.12581885
Generation:70525, Loss = 0.10492519
Generation:70550, Loss = 0.12197348
Generation:70575, Loss = 0.

Generation:75625, Loss = 0.11030102
Generation:75650, Loss = 0.12202504
Generation:75675, Loss = 0.109730825
Generation:75700, Loss = 0.11668501
Generation:75725, Loss = 0.10942561
Generation:75750, Loss = 0.12917118
Generation:75775, Loss = 0.12765695
Generation:75800, Loss = 0.11825923
Generation:75825, Loss = 0.10663062
Generation:75850, Loss = 0.1108398
Generation:75875, Loss = 0.1206087
Generation:75900, Loss = 0.106118776
Generation:75925, Loss = 0.10096002
Generation:75950, Loss = 0.120155774
Generation:75975, Loss = 0.12855518
Generation:76000, Loss = 0.12303672
Generation:76025, Loss = 0.12957636
Generation:76050, Loss = 0.111623615
Generation:76075, Loss = 0.114699185
Generation:76100, Loss = 0.116042316
Generation:76125, Loss = 0.124619156
Generation:76150, Loss = 0.13760462
Generation:76175, Loss = 0.11387256
Generation:76200, Loss = 0.11277699
Generation:76225, Loss = 0.10953866
Generation:76250, Loss = 0.10893876
Generation:76275, Loss = 0.12850654
Generation:76300, Loss 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.grid()
plt.plot(loss_vec)
#plt.xlim(0,25)
plt.ylim(0,2)
ax.set_xlabel('iteration')
ax.set_ylabel('loss')
plt.savefig(str(hidden_size)+'.png')

In [ ]:
# 重みの保存
saver = tf.train.Saver()
saver.save(sess, './model.ckpt')


In [ ]:
'''
# ボール座標とスティックの傾きプロット
by_max=IN_train.max(axis=0)[1]
by_min=IN_train.min(axis=0)[1]

ball_y = np.array(range(int(by_min)-3,int(by_max)+3))
for i in range(len(ball_y)):
    
    ball = np.array([[6.0, ball_y[i]/10.0]])
    
    print(ball)
    joy = sess.run(final_output, feed_dict={INPUT: ball})
    plt.scatter(ball[0][1],ball[0][0], s=200, c='orange')
    
    plt.quiver(0,0,-joy[0][0],-joy[0][1],angles="xy",scale_units="xy",scale=1)
    #plt.quiver(0,0,-0.6,5,angles="xy",scale_units="xy",scale=1)
    print(joy)
    plt.savefig(str(hidden_size)+'_img'+'.png')
    '''

In [ ]:
# 重みのrestore
predicts_sess = tf.Session()
ckpt = tf.train.get_checkpoint_state('./')
if ckpt == None:
    print('!!!!!!!WEIGHTS NOT FOUND!!!!!!')

saver.restore(predicts_sess, './model.ckpt')